In [1]:
import os

download_name = "worldcities_with_splits.csv.bz2"
if not os.path.exists(download_name):
    import requests
    response = requests.get(f"https://raw.githubusercontent.com/bzitko/nlp_repo/main/lectures/p03/{download_name}")
    with open(download_name, "wb") as fp:
        fp.write(response.content)
    response.close()
        
name = "worldcities_with_splits.csv"
if not os.path.exists(name):
    import bz2
    with open(download_name, 'rb') as bzf, open(name, 'wb') as fp:
        fp.write(bz2.decompress(bzf.read()))          

# Classifying Cities with a Multilayer Perceptron

## Imports

In [2]:
from argparse import Namespace
from collections import Counter
import json
import os
import string

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

## Data Vectorization classes

### The Vocabulary

In [3]:
class Vocabulary(object):
    def __init__(self, token_to_idx=None, add_unk=True, unk_token="<UNK>"):
        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx

        self._idx_to_token = {idx: token 
                              for token, idx in self._token_to_idx.items()}
        
        self._add_unk = add_unk
        self._unk_token = unk_token
        
        self.unk_index = -1
        if add_unk:
            self.unk_index = self.add_token(unk_token) 
        
    def to_serializable(self):
        return {'token_to_idx': self._token_to_idx, 
                'add_unk': self._add_unk, 
                'unk_token': self._unk_token}

    @classmethod
    def from_serializable(cls, contents):
        return cls(**contents)

    def add_token(self, token):
        try:
            index = self._token_to_idx[token]
        except KeyError:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
    
    def add_many(self, tokens):
        return [self.add_token(token) for token in tokens]

    def lookup_token(self, token):
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

    def lookup_index(self, index):
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]

    def __str__(self):
        return f"<Vocabulary(size={len(self)})>"

    def __len__(self):
        return len(self._token_to_idx)

### The Vectorizer

In [15]:
class CityVectorizer(object):
    def __init__(self, city_vocab, country_vocab):
        self.city_vocab = city_vocab
        self.country_vocab = country_vocab

    def vectorize(self, city):
        vocab = self.city_vocab
        one_hot = np.zeros(len(vocab), dtype=np.float32)
        for token in city:
            one_hot[vocab.lookup_token(token)] = 1

        return one_hot

    @classmethod
    def from_dataframe(cls, city_df):
        city_vocab = Vocabulary(unk_token="@")
        country_vocab = Vocabulary(add_unk=False)

        for index, row in city_df.iterrows():
            for letter in row.city:
                city_vocab.add_token(letter)
            country_vocab.add_token(row.country)

        return cls(city_vocab, country_vocab)

    @classmethod
    def from_serializable(cls, contents):
        city_vocab = Vocabulary.from_serializable(contents['city_vocab'])
        country_vocab =  Vocabulary.from_serializable(contents['country_vocab'])
        return cls(city_vocab=city_vocab, country_vocab=country_vocab)

    def to_serializable(self):
        return {'city_vocab': self.city_vocab.to_serializable(),
                'country_vocab': self.country_vocab.to_serializable()}

### The Dataset

In [16]:
class CityDataset(Dataset):
    def __init__(self, city_df, vectorizer):
        self.city_df = city_df
        self._vectorizer = vectorizer

        self.train_df = self.city_df[self.city_df.split=='train']
        self.train_size = len(self.train_df)

        self.val_df = self.city_df[self.city_df.split=='val']
        self.validation_size = len(self.val_df)

        self.test_df = self.city_df[self.city_df.split=='test']
        self.test_size = len(self.test_df)

        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.validation_size),
                             'test': (self.test_df, self.test_size)}

        self.set_split('train')
        
        # Class weights
        class_counts = city_df.country.value_counts().to_dict()
        def sort_key(item):
            return self._vectorizer.country_vocab.lookup_token(item[0])

        sorted_counts = sorted(class_counts.items(), key=sort_key)
        frequencies = [count for _, count in sorted_counts]
        self.class_weights = 1.0 / torch.tensor(frequencies, dtype=torch.float32)

    @classmethod
    def load_dataset_and_make_vectorizer(cls, city_csv):
        city_df = pd.read_csv(city_csv)
        train_city_df = city_df[city_df.split=='train']
        return cls(city_df, CityVectorizer.from_dataframe(train_city_df))

    @classmethod
    def load_dataset_and_load_vectorizer(cls, city_csv, vectorizer_filepath):
        city_df = pd.read_csv(city_csv)
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(city_df, vectorizer)

    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        with open(vectorizer_filepath) as fp:
            return CityVectorizer.from_serializable(json.load(fp))

    def save_vectorizer(self, vectorizer_filepath):
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)

    def get_vectorizer(self):
        return self._vectorizer

    def set_split(self, split="train"):
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        row = self._target_df.iloc[index]

        city_vector = self._vectorizer.vectorize(row.city)

        country_index = self._vectorizer.country_vocab.lookup_token(row.country)

        return {'x_city': city_vector,
                'y_country': country_index}

    def get_num_batches(self, batch_size):
        return len(self) // batch_size

    
def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"): 
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

## The Model: CityClassifier

In [55]:
class CityClassifier(nn.Module):

    def __init__(self, input_dim, hidden_dim, output_dim):
        super(CityClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x_in, apply_softmax=False):
        intermediate_vector = F.relu(self.fc1(x_in))
        prediction_vector = self.fc2(intermediate_vector)

        if apply_softmax:
            prediction_vector = F.softmax(prediction_vector, dim=1)

        return prediction_vector


class MultilayerPerceptron(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim=2, output_dim=3, 
                 num_hidden_layers=1, hidden_activation=torch.nn.ReLU):
        super(MultilayerPerceptron, self).__init__()
        self.module_list = torch.nn.ModuleList()
        
        interim_input_dim = input_dim
        interim_output_dim = hidden_dim
        
        for _ in range(num_hidden_layers):
            self.module_list.append(torch.nn.Linear(interim_input_dim, interim_output_dim))
            self.module_list.append(hidden_activation())
            interim_input_dim = interim_output_dim

        self.fc_final = nn.Linear(interim_input_dim, output_dim)
    
    def forward(self, x, apply_softmax=False):
        for module in self.module_list:
            x = module(x)
        output = self.fc_final(x)
        if apply_softmax:
            output = F.softmax(output, dim=1)
        return output

## Training Routine

### Helper functions

In [18]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}

def update_train_state(args, model, train_state):
    # Save one model at least
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # Save model if performance improved
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]

        # If loss worsened
        if loss_t >= train_state['early_stopping_best_val']:
            # Update step
            train_state['early_stopping_step'] += 1
        # Loss decreased
        else:
            # Save the best model
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])

            # Reset early stopping step
            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

#### general utilities

In [19]:
def set_seed_everywhere(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

### Settings and some prep work

In [36]:
args = Namespace(
    # Data and path information
    city_csv="worldcities_with_splits.csv",
    vectorizer_file="vectorizer.json",
    model_state_file="model.pth",
    save_dir=".",
    # Model hyper parameters
    hidden_dim=700,
    # Training  hyper parameters
    seed=1337,
    num_epochs=100,
    early_stopping_criteria=5,
    learning_rate=0.001,
    batch_size=64,
    # Runtime options
    cuda=True,
    reload_from_files=False,
    expand_filepaths_to_save_dir=True,
)

if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir, args.vectorizer_file)
    args.model_state_file = os.path.join(args.save_dir, args.model_state_file)
    
    print("Expanded filepaths: ")
    print(f"\t{args.vectorizer_file}")
    print(f"\t{args.model_state_file}")
    
# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else "cpu")
    
print(f"Using CUDA: {args.cuda}")


# Set seed for reproducibility
set_seed_everywhere(args.seed)

# handle dirs
handle_dirs(args.save_dir)

Expanded filepaths: 
	./vectorizer.json
	./model.pth
Using CUDA: False


### Initializations

In [56]:
if args.reload_from_files:
    # training from a checkpoint
    print("Reloading!")
    dataset = CityDataset.load_dataset_and_load_vectorizer(args.city_csv,   
                                                           args.vectorizer_file)
else:
    # create dataset and vectorizer
    print("Creating fresh!")
    dataset = CityDataset.load_dataset_and_make_vectorizer(args.city_csv)
    dataset.save_vectorizer(args.vectorizer_file)
    
vectorizer = dataset.get_vectorizer()
classifier = CityClassifier(input_dim=len(vectorizer.city_vocab), 
                            hidden_dim=args.hidden_dim, 
                            output_dim=len(vectorizer.country_vocab))

classifier = MultilayerPerceptron(input_dim=len(vectorizer.city_vocab), 
                                  num_hidden_layers=2,
                                  hidden_dim=args.hidden_dim, 
                                  output_dim=len(vectorizer.country_vocab))

classifier


Creating fresh!


MultilayerPerceptron(
  (module_list): ModuleList(
    (0): Linear(in_features=211, out_features=700, bias=True)
    (1): ReLU()
    (2): Linear(in_features=700, out_features=700, bias=True)
    (3): ReLU()
  )
  (fc_final): Linear(in_features=700, out_features=174, bias=True)
)

### Training loop

In [57]:
classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)

    
loss_func = nn.CrossEntropyLoss(dataset.class_weights)
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                 mode='min', factor=0.5,
                                                 patience=1)

train_state = make_train_state(args)

epoch_bar = tqdm(desc='training routine', 
                 total=args.num_epochs,
                 position=0)

dataset.set_split('train')
train_bar = tqdm(desc='split=train',
                 total=dataset.get_num_batches(args.batch_size), 
                 position=1, 
                 leave=True)
dataset.set_split('val')
val_bar = tqdm(desc='split=val',
               total=dataset.get_num_batches(args.batch_size), 
               position=1, 
               leave=True)

try:
    for epoch_index in range(args.num_epochs):
        train_state['epoch_index'] = epoch_index

        # Iterate over training dataset

        # setup: batch generator, set loss and acc to 0, set train mode on

        dataset.set_split('train')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        classifier.train()

        for batch_index, batch_dict in enumerate(batch_generator):
            # the training routine is these 5 steps:

            # --------------------------------------
            # step 1. zero the gradients
            optimizer.zero_grad()

            # step 2. compute the output
            y_pred = classifier(batch_dict['x_city'])

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_country'])
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # step 4. use loss to produce gradients
            loss.backward()

            # step 5. use optimizer to take gradient step
            optimizer.step()
            # -----------------------------------------
            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_country'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

            # update bar
            train_bar.set_postfix(loss=running_loss, acc=running_acc, 
                            epoch=epoch_index)
            train_bar.update()

        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)

        # Iterate over val dataset

        # setup: batch generator, set loss and acc to 0; set eval mode on
        dataset.set_split('val')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.
        running_acc = 0.
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):

            # compute the output
            y_pred =  classifier(batch_dict['x_city'])

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_country'])
            loss_t = loss.to("cpu").item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_country'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)
            val_bar.set_postfix(loss=running_loss, acc=running_acc, 
                            epoch=epoch_index)
            val_bar.update()

        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)

        train_state = update_train_state(args=args, model=classifier,
                                         train_state=train_state)

        scheduler.step(train_state['val_loss'][-1])

        if train_state['stop_early']:
            break

        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()
except KeyboardInterrupt:
    print("Exiting loop")


training routine:   0%|          | 0/100 [00:00<?, ?it/s]

split=train:   0%|          | 0/466 [00:00<?, ?it/s]

split=val:   0%|          | 0/98 [00:00<?, ?it/s]

Exiting loop


In [58]:
# compute the loss & accuracy on the test set using the best available model

classifier.load_state_dict(torch.load(train_state['model_filename']))

classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)
loss_func = nn.CrossEntropyLoss(dataset.class_weights)

dataset.set_split('test')
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred =  classifier(batch_dict['x_city'])
    
    # compute the loss
    loss = loss_func(y_pred, batch_dict['y_country'])
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # compute the accuracy
    acc_t = compute_accuracy(y_pred, batch_dict['y_country'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc


In [59]:
print(f"Test loss: {train_state['test_loss']};")
print(f"Test Accuracy: {train_state['test_acc']}")

Test loss: 5.109314627507152;
Test Accuracy: 19.653799019607842


### Inference

In [60]:
def predict_country(city, classifier, vectorizer):
    vectorized_city = vectorizer.vectorize(city)
    vectorized_city = torch.tensor(vectorized_city).view(1, -1)
    result = classifier(vectorized_city, apply_softmax=True)

    probability_values, indices = result.max(dim=1)
    index = indices.item()

    predicted_country = vectorizer.country_vocab.lookup_index(index)
    probability_value = probability_values.item()

    return {'country': predicted_country, 'probability': probability_value}


### Top-K Inference

In [63]:
def predict_topk_country(city, classifier, vectorizer, k=5):
    vectorized_city = vectorizer.vectorize(city)
    vectorized_city = torch.tensor(vectorized_city).view(1, -1)
    prediction_vector = classifier(vectorized_city, apply_softmax=True)
    probability_values, indices = torch.topk(prediction_vector, k=k)
    
    # returned size is 1,k
    probability_values = probability_values.detach().numpy()[0]
    indices = indices.detach().numpy()[0]
    
    results = []
    for prob_value, index in zip(probability_values, indices):
        country = vectorizer.country_vocab.lookup_index(index)
        results.append({'country': country, 
                        'probability': prob_value})
    
    return results

def topk_predictions(city, k=5):
    if k > len(vectorizer.country_vocab):
        print("Sorry! That's more than the # of nationalities we have.. defaulting you to max size :)")
        k = len(vectorizer.country_vocab)
    predictions = predict_topk_country(city, classifier, vectorizer, k=k)
    
    print(f"Top {k} predictions:")
    print("===================")
    for prediction in predictions:
        print(f"{city} -> {prediction['country']} (p={prediction['probability']:0.2f})")

In [68]:
topk_predictions("Beograd", 5)

Top 5 predictions:
Beograd -> Somalia (p=0.20)
Beograd -> Zimbabwe (p=0.15)
Beograd -> India (p=0.09)
Beograd -> Italy (p=0.06)
Beograd -> Colombia (p=0.03)


In [35]:
vec = dataset.get_vectorizer()
vec.city_vocab._token_to_idx

{'@': 0,
 'F': 1,
 'a': 2,
 'r': 3,
 'ā': 4,
 'h': 5,
 'K': 6,
 'b': 7,
 'u': 8,
 'l': 9,
 'T': 10,
 'q': 11,
 'n': 12,
 'Z': 13,
 'g': 14,
 'ū': 15,
 ' ': 16,
 'S': 17,
 'B': 18,
 'm': 19,
 'y': 20,
 'A': 21,
 'd': 22,
 'k': 23,
 'ō': 24,
 'P': 25,
 '-': 26,
 'e': 27,
 'ī': 28,
 'L': 29,
 's': 30,
 'G': 31,
 't': 32,
 'z': 33,
 'J': 34,
 'N': 35,
 'M': 36,
 'f': 37,
 'ẕ': 38,
 'C': 39,
 'ē': 40,
 'j': 41,
 '‘': 42,
 'H': 43,
 'ṯ': 44,
 '̲': 45,
 'Q': 46,
 'o': 47,
 'w': 48,
 'ç': 49,
 'v': 50,
 'ë': 51,
 'i': 52,
 'R': 53,
 'p': 54,
 'D': 55,
 'V': 56,
 'c': 57,
 'E': 58,
 'O': 59,
 'Y': 60,
 'I': 61,
 '’': 62,
 'è': 63,
 'ï': 64,
 'é': 65,
 'â': 66,
 'U': 67,
 'í': 68,
 'x': 69,
 'X': 70,
 '.': 71,
 'á': 72,
 'ú': 73,
 'Á': 74,
 'ñ': 75,
 'ó': 76,
 'ü': 77,
 'W': 78,
 'ö': 79,
 'ä': 80,
 'ş': 81,
 'ğ': 82,
 'ı': 83,
 'ə': 84,
 'Ş': 85,
 'İ': 86,
 'Ə': 87,
 'É': 88,
 'à': 89,
 'ž': 90,
 'ć': 91,
 'š': 92,
 'č': 93,
 'Č': 94,
 'ã': 95,
 'ô': 96,
 'ê': 97,
 'õ': 98,
 'Ó': 99,
 'Í': 100,

In [40]:
vec.country_vocab._token_to_idx

{'Afghanistan': 0,
 'Albania': 1,
 'Algeria': 2,
 'Andorra': 3,
 'Angola': 4,
 'Argentina': 5,
 'Armenia': 6,
 'Australia': 7,
 'Austria': 8,
 'Azerbaijan': 9,
 'Bangladesh': 10,
 'Belarus': 11,
 'Belgium': 12,
 'Belize': 13,
 'Benin': 14,
 'Bhutan': 15,
 'Bolivia': 16,
 'Bosnia And Herzegovina': 17,
 'Botswana': 18,
 'Brazil': 19,
 'Brunei': 20,
 'Bulgaria': 21,
 'Burkina Faso': 22,
 'Burundi': 23,
 'Cabo Verde': 24,
 'Cambodia': 25,
 'Cameroon': 26,
 'Canada': 27,
 'Central African Republic': 28,
 'Chad': 29,
 'Chile': 30,
 'China': 31,
 'Colombia': 32,
 'Congo (Brazzaville)': 33,
 'Congo (Kinshasa)': 34,
 'Costa Rica': 35,
 'Croatia': 36,
 'Cuba': 37,
 'Cyprus': 38,
 'Czechia': 39,
 "Côte d'Ivoire": 40,
 'Denmark': 41,
 'Dominican Republic': 42,
 'Ecuador': 43,
 'Egypt': 44,
 'El Salvador': 45,
 'Equatorial Guinea': 46,
 'Eritrea': 47,
 'Estonia': 48,
 'Ethiopia': 49,
 'Faroe Islands': 50,
 'Fiji': 51,
 'Finland': 52,
 'France': 53,
 'French Guiana': 54,
 'Gabon': 55,
 'Georgia': 56